# Tabular-Playground Compitution 

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
submession = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
print("The Shape of Training Set : {}".format(train.shape))
print("The Shape of Testing Set : {}".format(test.shape))
print("The Shape of Submession Set : {}".format(submession.shape))




## Checking Missvalues

In [ ]:
train.isnull().sum().sort_values(ascending =False)

In [ ]:
test.isnull().sum().sort_values(ascending =False)

In [ ]:
submession.isnull().sum().sort_values(ascending =False)

In [ ]:
test.head()

# Plot the Target

In [ ]:
sns.displot(train.loss)

In [ ]:
sns.displot(submession.loss)

In [ ]:
submession.loss.value_counts(ascending =False)

## Analys the Correlation

In [ ]:
fig = plt.figure(figsize=(15,12))
sns.heatmap(train.corr())

In [ ]:
corr_train=train.corr()
corr_train = corr_train['loss'].sort_values(ascending=False).round(2)
corr_train

## The most correlated Features (Above 0)

In [ ]:
corr_train[corr_train >0]

## Drop the Id-Feature

In [ ]:
# No Relevant for the Work
X = train.drop('id', axis = 1)
X_test = test.drop('id', axis = 1)


In [ ]:
X.shape

## Analyse the Data Types

In [ ]:
X.dtypes.value_counts()

In [ ]:
X.head()
test.head()

In [ ]:
X = X.drop('loss',axis = 1)
print("Shape of the dependent Features {}".format(X.shape))
y = train['loss']


In [ ]:
print("The Shape of X : {}".format(X.shape))
print("The Shape of X_test : {}".format(X_test.shape))


## Split The Dataset

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size = 0.3, random_state = 1)


## Dimensionality reduction With PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

#The best 50 Features
pca = Pipeline([('scale', MinMaxScaler()), ('pca', PCA(49, random_state=0))
]).fit(X_train)
X_train_pca = pca.transform(X_train)
X_val_pca = pca.transform(X_val)
X_test =pca.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=123).fit(X_train, y_train)

In [ ]:
Features_importance = pd.DataFrame([(col,coef) for col , coef in zip(X.columns,dt.feature_importances_)], 
             columns = ['feature', 'importance']).set_index('feature').sort_values('importance',ascending = False)
Features_importance

## Select the best 50 Column

In [ ]:
Features_importance = Features_importance[Features_importance.values > 0.01]
Features_importance.shape

In [ ]:
Features_importance.index

In [ ]:
X_RF = X[Features_importance.index]
X_test_RF = test.drop (columns = ['id'])
X_test_RF = X_test_RF[Features_importance.index]
X_RF.shape,X_test_RF.shape

## Model I
- `GradientBoostingRegressor`
- I use Randomforest and PCA to select the best Features, so the randomForest perform better than the PCA Methode

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

#gbrt = GradientBoostingRegressor(random_state=123, learning_rate=0.1,n_estimators=100).fit(X_train_pca, y_train)
#gbrt_pred=gbrt.predict(X_val_pca)




In [ ]:

scaler = StandardScaler()
scaler.fit(X_RF)
X_RF = scaler.transform(X_RF)
X_test_RF = scaler.transform(X_test_RF)
X_train_RF,X_val_RF,y_train,y_val=train_test_split(X_RF,y,test_size = 0.3, random_state = 1)




## Use RandomForest_Model for Features Selection

In [ ]:
#gbrt_RF = GradientBoostingRegressor(random_state=123, learning_rate=0.1,n_estimators=100).fit(X_train_RF, y_train)
#xgb_pred_RF=gbrt_RF.predict(X_val_RF)

## Use The Gridsearch Methode to improve the model 

In [ ]:
gb = GradientBoostingRegressor(n_estimators=100, random_state=0)
search_space = {
'max_depth' : [4, 8], # keep trees small to avoid overfitting

'min_samples_leaf' : [4, 6],

'learning_rate' : [0.01,0.1, 0.5, 1]}
    
gbrt_RF = GridSearchCV(gb, search_space, cv=5).fit(X_train_RF,y_train)
gbrt_pred_RF=gbrt_RF.predict(X_val_RF)

## Model II 
- `XGBRegressor`

In [ ]:
import xgboost


xgb=xgboost.XGBRegressor(random_state=24)
search_space={
    'max_depth' : [4, 6], # keep trees small to avoid overfitting

    'n_estimators' : [100, 500],

    'learning_rate' : [0.01,0.1, 0.5, 1]}
    
xgb_grid = GridSearchCV(xgb, search_space, cv=5).fit(X_train_RF,y_train)
xgb_pred=xgb_grid.predict(X_val_RF)

## Model III
- `LinearRegression`

In [ ]:


from sklearn.linear_model import LinearRegression
lm = LinearRegression()

param_grid =  {
'normalize' : [True, False],
'copy_X' :[True,False],
'n_jobs': [1,10]}

        
lm_grid = GridSearchCV(lm ,param_grid,cv=5).fit(X_train_RF,y_train)
lm_pred_RF =lm_grid.predict(X_val_RF)

## Model Evaluation

In [ ]:
from sklearn.metrics import r2_score

#print("gbrt.score: {}".format(gbrt.score(X_val_pca,y_val)))
#print("gbrt_r2_score: {}".format(r2_score(y_val,gbrt_pred)))


#print("xgb.score: {}".format(gbrt.score(X_val_pca,y_val)))
#print("xgb_r2_score: {}".format(r2_score(y_val,xgb_pred)))

In [ ]:
from sklearn.metrics import mean_squared_error

rmse_gbrt = mean_squared_error(y_val,gbrt_pred_RF, squared = False)

rmse_xgb_RF = mean_squared_error(y_val,xgb_pred_RF, squared = False)
rmse_lm_RF = mean_squared_error(y_val,lm_pred_RF, squared = False)


#print("rmse_xgb :",rmse_xgb)
print("rmse_gbrt :",rmse_gbrt)
print("rmse_gbrt_RF :",rmse_xgb_RF)
print("rmse_gbrt_RF :",rmse_lm_RF)


In [ ]:
X_train.shape,X_test.shape

In [ ]:
Sub_final_2 = gbrt_RF.predict(X_test)
Sub_final_2 = np.clip(Sub_final_2,y_train.min(),y_train.max()) 
Sub_final_2

In [ ]:
Sub_final_3 = xgb_grid.predict(X_test)
Sub_final_3 = np.clip(Sub_final_3,y_train.min(),y_train.max()) 
Sub_final_3

In [ ]:
Sub_final_4 = lm_grid.predict(X_test_RF)
Sub_final_4 = np.clip(Sub_final_4,y_train.min(),y_train.max()) 
Sub_final_4

## Ensemble methods

I try the opinions to combine various models with using `VotingRegressor`to make the final decision

In [ ]:
from sklearn.ensemble import VotingRegressor

majority_rules = VotingRegressor([('gbrt', gbrt_RF.best_estimator_),('lm', lm_grid.best_estimator_),('xgb', xgb_grid.best_estimator_)]).fit(X_train_RF,y_train)


max_probabilities = VotingRegressor([('gbrt', gbrt_RF.best_estimator_),('lm', lm_grid.best_estimator_),('xgb', xgb_grid.best_estimator_)]).fit(X_train_RF, y_train)


In [ ]:
Final_model = max_probabilities.predict(X_test_RF)
Final_model = np.clip(Final_model,y_train.min(),y_train.max()) 
Final_model

In [ ]:
grt_model=pd.DataFrame({'loss': Sub_final_2})
xgb_model=pd.DataFrame({'loss': Sub_final_3})
lm_model=pd.DataFrame({'loss': Sub_final_4})
Model = pd.DataFrame({'loss': Final_model})
Sub_final_2= pd.concat([submession['id'].reset_index(drop=True), gbrt_model], axis=1)
Sub_final_3= pd.concat([submession['id'].reset_index(drop=True), xgb_model], axis=1)
Sub_final_4= pd.concat([submession['id'].reset_index(drop=True), lm_model], axis=1)
Sub_model = pd.concat([submession['id'].reset_index(drop=True), Model], axis=1)

In [ ]:
import csv

Sub_final_2.to_csv('./Submission_gbrt.csv',index=False)
Sub_final_3.to_csv('./Submission_xgb.csv',index=False)
Sub_final_4.to_csv('./Submission_lm.csv',index=False)

In [ ]:
pd.read_csv("./Submission_xgb.csv").head()